# Tarefa 02: O Algoritmo Random Forest (Floresta Aleatória)

## 1. Passo a Passo do Algoritmo Random Forest (RF)

O funcionamento da **Random Forest** é um processo iterativo que visa aumentar a diversidade entre os modelos. Os estágios principais são:

* **Amostragem Bootstrap:** Assim como no Bagging, o algoritmo cria múltiplos subconjuntos de dados sorteando linhas aleatoriamente com reposição.
* **Seleção Aleatória de Recursos (Feature Selection):** Em cada nó de cada árvore, em vez de buscar a melhor divisão entre todas as variáveis, o algoritmo sorteia um subconjunto aleatório de colunas (features).
* **Criação das Árvores (Modelagem):** Uma Árvore de Decisão é treinada para cada amostra. Devido à restrição de variáveis do passo anterior, cada árvore "enxerga" o problema de um ângulo diferente.
* **Crescimento Máximo:** Geralmente, as árvores na Random Forest são deixadas crescer profundamente, permitindo que cada uma capture padrões específicos da sua amostra.
* **Votação/Média (Agregação):** As previsões individuais de todas as árvores são combinadas para gerar o resultado final (Votação para classificação ou Média para regressão).



---

## 2. Explicação Conceitual

A **Random Forest** pode ser vista como uma evolução técnica do Bagging. Ela utiliza múltiplas árvores de decisão, mas adiciona uma camada extra de aleatoriedade na seleção de variáveis (*features*). 

Como cada árvore analisa dados e sintomas distintos, o diagnóstico final torna-se muito mais sólido e diversificado. Essa abordagem impede que o modelo se torne "viciado" em uma única variável predominante que, embora pareça óbvia, poderia levar a conclusões incorretas se analisada isoladamente. Na prática, a Random Forest descorrelaciona as árvores, garantindo que o erro de uma não seja repetido por todas as outras.

---

## 3. Diferença Fundamental: Bagging vs. Random Forest

| Característica | Bagging | Random Forest |
| :--- | :--- | :--- |
| **Amostragem de Linhas** | Bootstrap (com reposição) | Bootstrap (com reposição) |
| **Amostragem de Colunas** | Considera todas as variáveis | Considera um subconjunto aleatório |
| **Independência** | Árvores podem ser parecidas | Árvores são forçadas a ser diferentes |
| **Aplicação Principal** | Redução de Variância | Redução de Variância e Descorrelação |

------------

## 4. Implementação em Python

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
# Preparação dos dados
df_fechamento = pd.read_csv('dados_ativos_fechamento.csv', index_col=0, parse_dates=True)
df_ret = df_fechamento.pct_change().dropna()

# Alvo: VALE3 subiu hoje? (1 para sim, 0 para não)
X = df_ret.drop(columns=['VALE3.SA']) # Usamos PETR e ITUB para prever VALE
y = (df_ret['VALE3.SA'] > 0).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Implementando a Random Forest
# Aqui o RandomForestClassifier já faz o Bootstrap, Feature Selection e Agregação
modelo_rf = RandomForestClassifier(
    n_estimators=100,      # Modelagem com 100 Decision Trees
    max_features='sqrt',   # Feature Selection: raiz quadrada das colunas
    bootstrap=True,        # Ativa o Bootstrap
    random_state=42
)

# Treinamento
modelo_rf.fit(X_train, y_train)

# Resultados
previsoes = modelo_rf.predict(X_test)
print("Relatório de Classificação - Random Forest:")
print(classification_report(y_test, previsoes))

# Importância das Variáveis
importancias = pd.Series(modelo_rf.feature_importances_, index=X.columns)
print("\nVariáveis que mais influenciaram a floresta:")
print(importancias.sort_values(ascending=False))

Relatório de Classificação - Random Forest:
              precision    recall  f1-score   support

           0       0.52      0.60      0.55        72
           1       0.57      0.49      0.52        78

    accuracy                           0.54       150
   macro avg       0.54      0.54      0.54       150
weighted avg       0.54      0.54      0.54       150


Variáveis que mais influenciaram a floresta:
PETR4.SA    0.503651
ITUB4.SA    0.496349
dtype: float64


---------------